In [76]:
import numpy as np 
import torch
from data.wrappers import load_data
from model.odegpvae import ODEGPVAE

device = 'cpu'
data_root = 'data/'
task = 'mnist'


In [77]:
########### data ############ 
trainset, testset = load_data(data_root, task, plot=True)


########### model ###########
odegpvae = ODEGPVAE(q=8,n_filt=16).to(device)

In [78]:
# ########### train ###########
Nepoch = 500
optimizer = torch.optim.Adam(odegpvae.parameters(),lr=1e-3)

In [79]:
minibatch = next(iter(trainset)).to(device)

In [80]:
X = minibatch

In [81]:
[N,T,nc,d,d] = X.shape

In [82]:
X.shape

torch.Size([25, 16, 1, 28, 28])

In [83]:
X[:,0].shape

torch.Size([25, 1, 28, 28])

In [84]:
h = odegpvae.encoder(X[:,0])

In [85]:
qz0_m, qz0_logv = odegpvae.fc1(h), odegpvae.fc2(h)

In [86]:
q = qz0_m.shape[1]//2

In [87]:
eps   = torch.randn_like(qz0_m, dtype=odegpvae.ode_model.svgp.dtype)
z0= qz0_m + eps*torch.exp(qz0_logv) # N,2q

In [88]:
logp0 = odegpvae.mvn.log_prob(eps)

In [89]:
z0.shape

torch.Size([25, 16])

In [90]:
gp_draw = odegpvae.ode_model.svgp.draw_posterior_function()

In [91]:
odef = lambda t,x: gp_draw(x)

In [92]:
t  = 0.1 * torch.arange(T,dtype=torch.float).to(z0.device)

In [93]:
gp_draw = odegpvae.ode_model.svgp.draw_posterior_function() 

In [94]:
oderhs = lambda t, x: odegpvae.ode_model.svgp.ode_rhs(t,x,gp_draw)

In [95]:
from torchdiffeq import odeint

In [96]:
zt,logp = odeint(oderhs, z0, t,method="dopri5")

AttributeError: 'SVGP' object has no attribute 'ode_rhs'

In [60]:
from model.helpers import integrate
integrate(odef, z0, t)

RuntimeError: The size of tensor a (16) must match the size of tensor b (8) at non-singleton dimension 1